## no se observa la estructura normal descrita en la documentacion de Tweeter
- No se encuentra retweet_status 
- No se observa screen_name
- No se observa presencia de el tag entities

In [485]:
import os
import string
import pandas as pd

import json
from collections import Counter
import re
from glob import glob
from datetime import datetime
from matplotlib import pyplot as plt
from time import gmtime, mktime, strptime
import warnings
warnings.filterwarnings('ignore')

In [444]:
# function to get unique values
def unique(list1):
 
    # initialize a null list
    unique_list = []
     
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    # print list
    return unique_list

## Load json file

In [335]:
DIR = 'C:/Users/gufernandez/Documents/tweets.json/farmers-protest-tweets-2021-2-4.json'
#lfiles = glob(DIR + "farmers-protest-tweets-2021-2-4.json")

In [336]:
tweets_data_path = DIR

In [337]:
tweets_data = []
tweets_file = open(tweets_data_path, "r")

### read line by line

In [338]:
for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue

In [339]:
# Elimina posibles tweet sin user
for tweet in tweets_data:
    try:
        user = tweet['user']
    except:
        tweets_data.remove(tweet)
 
print(len(tweets_data))

117407


### Extraer info de importancia para la solución

In [434]:
rows_list = []
for tweet in tweets_data:
    
    author = ""
    rtauthor = ""
    
    ## Extrae la información de usuario autor y el retweet, si es un retweet
    try:
        
        author = tweet['user']['username']
        rtauthor = tweet['retweeted_status']['user']['username']
        
    except:

        ## Si no es un retweet obtiene la información del usuario autor solamente
        try:
            author = tweet['user']['username']
        except:
            continue
    
    # Se extrae la data desde el tweet
    create_date = datetime.strptime(str(tweet['user']['created'])[0:10], '%Y-%M-%d').date()
    followers = tweet['user']['followersCount']
    replyCount = tweet['replyCount']
    retweetedTweet = tweet['retweetedTweet']
    retweetCount = tweet['retweetCount']
    
    # fue necesario extraer los hashtag desde el content del tweet ya que no esta presente el tag de 'entities' donde se encuentra
    # la info de los hashtag de un tweet
    hashtags_list = re.split(r',|\n| ', str(tweet['content']))
    hashtags = ""
    
    # se considerara hashtag todo aquel texto que comiece con el simbolo '#'
    for element in hashtags_list:
        
        regexp = re.compile(r'^#')
        
        if regexp.search(element):
            if str(element) == '[]':
                element = ''
            hashtags.append(element)
    
    
    dict1 = {}
    dict1.update({'author': author, 'create_date': create_date, 'replyCount': replyCount, 'hashtags':hashtags , 'retweetedTweet': retweetedTweet, 'retweetCount': retweetCount , 'followers': followers})
    rows_list.append(dict1)

In [435]:
tweets = pd.DataFrame(rows_list)

In [436]:
tweets

,author,create_date,replyCount,hashtags,retweetedTweet,retweetCount,followers
0,ArjunSinghPanam,2009-01-06,0,"[#ModiDontSellFarmers, #FarmersProtest, #FreeN...",None,0,603
1,PrdeepNain,2021-01-29,0,"[#FarmersProtest, #ModiIgnoringFarmersDeaths, ...",None,0,14
2,parmarmaninder,2012-01-27,0,[#FarmersProtest],None,0,72
3,anmoldhaliwal,2010-01-28,0,"[#farmersprotest, #NoFarmersNoFood]",None,0,51
4,KotiaPreet,2021-01-08,0,"[#KisanEktaMorcha, #FarmersProtest, #NoFarmers...",None,0,5
...,...,...,...,...,...,...,...
117402,rickyrickstir,2012-01-25,0,"[#FarmersProtest, #KisanAndolan, #KisaanMajdoo...",None,0,29
117403,PunjabTak,2018-01-06,0,"[#FarmersProtest, #KisanAandolan, #FarmersFury...",None,0,354
117404,ish_kayy,2012-01-29,0,"[#MahapanchayatRevolution, #FarmersProtest]",None,65,137
117405,TV9Bharatvarsh,2018-01-12,0,"[#FarmersProtest, #MSP, #FarmBill2020, #Singhu...",None,1,130257


### 1 .- Obtenga cuales fueron los 10 tweets más retweeted y a que usuarios pertenecen esos tweets.

In [287]:
top_ten_retweet_user = tweets[['author','retweetCount']].sort_values(by='retweetCount', ascending=False).head(10)

In [288]:
top_ten_retweet_user

,author,retweetCount
111329,RakeshTikaitBKU,7723
7645,dhruv_rathee,6164
89780,rupikaur_,4673
88911,amaanbali,3742
111556,jedijasmin_,3332
64492,rupikaur_,3230
108072,RaviSinghKA,3182
60721,sherryontopp,3057
29510,sherryontopp,3040
24160,sherryontopp,2622


### 2.-  Los top 10 users en función a la cantidad de tweets que emitieron (no considerar retweets).

In [480]:
top_ten_tweet_user = tweets[['author']].value_counts().head(10)

In [481]:
for i, row in top_ten_tweet_user.iteritems():
  print(str(i), row)

('jot__b',) 1019
('rebelpacifist',) 850
('MaanDee08215437',) 830
('Gurpreetd86',) 636
('GurmVicky',) 597
('shells_n_petals',) 576
('preetysaini321',) 573
('ish_kayy',) 515
('KaurDosanjh1979',) 512
('DigitalKisanBot',) 490


## 3.- Obtenga los 10 días donde hay más tweets, muestre el día, la cantidad de tweets y todos los hastags que se usaron ese día

In [437]:
top_ten_tweet_by_day = tweets['create_date'].value_counts().head(10)

In [438]:
list_days = top_ten_tweet_by_day

In [439]:
type(list_days)

pandas.core.series.Series

In [476]:
list_hashtag = []
top_ten_tweets_by_day = []
hashtag_values = []

# loop sobre los 10 dias con mas tweets
for index, value in list_days.items():

    # obtengo los hashtags utilizados en el día especificado en el loop
    hashtags_on_date = tweets['hashtags'][(tweets['create_date']==index)]
    
    for hashtag_list_on_date in hashtags_on_date:
        hashtag_values += (hashtag_list_on_date)   
             
    final_list_hashtag = unique(hashtag_values)        
            
    top_ten_tweets_by_day.append([index, value,final_list_hashtag])


In [479]:
top_ten_tweets_by_day[0]

[datetime.date(2020, 1, 29),
 5192,
 ['#FarmersProtest',
  '#ModiDontSellFarmers',
  '#BJP',
  '#Unnao',
  '#Unnao_ki_Beti',
  '#WestBengal',
  '#WestBengalElection2021',
  '#Kolkata',
  '#AssamBachao',
  '#Assamese',
  '#GodiMedia',
  '#StandWithFarmers',
  '#मोदी_मतलब_देश_चौपट',
  '#Tractor2Twitter',
  '#Rajasthan',
  '#India',
  '#Dehli',
  '#ModiIgnoringFarmersDeath',
  '#ReleaseNodeepKaur',
  '#ਖ਼ੁਦਮੁਖਤਿਆਰੀ',
  '#कब_मिलेगा_न्याय',
  '#कब_होगा_काउंसलिंग',
  '#किसान_एकता_जिंदाबाद',
  '#ਮਾਂ_ਬੋਲੀ_ਪੰਜਾਬੀ_ਜ਼ਿੰਦਾਬਾਦ',
  '#FarmersMakeIndia',
  '#FarmersProtests',
  '#FarmersAboveReligiousHate',
  '#farmersprotest',
  '#FreeNodeepKaur',
  '#Gujarat',
  '#Maharashtra',
  '#Punjab',
  '#Farmersprotest',
  '#ReleaseMohinderKhalsa',
  '#kisaanmajdoorektazindabad',
  '#WeUnitedForFarmers',
  '#HumanRights4farmers',
  '#TakeBackFarmLaws',
  '#RepealBlackLaws',
  '#SpeakUpForFarmers',
  '#DalitLivesMatter',
  '#FarmersProtests2020',
  '#FarmerProtest',
  '#ModiPlanningFarmerGenocide',
  '#Fencing

## 4.- Los top 10 hashtags más usados.

In [491]:
hashtags = tweets['hashtags']
hashtag_words = []

for hashtag_list in hashtags:
    hashtag_words += (hashtag_list_on_date)   

# genero la lista del top 10 de hashtags mas usados
c = Counter(hashtag_values)

In [492]:
print ("",c.most_common(10))    

 [('#FarmersProtest', 27863), ('#ReleaseDetainedFarmers', 1858), ('#MahapanchayatRevolution', 1560), ('#FarmersMakeIndia', 1472), ('#RepealOnlyWayAhead', 1465), ('#Pagdi_Sambhal_Jatta', 1083), ('#IndiaBeingSilenced', 1052), ('#शहीद_जवान_शहीद_किसान', 844), ('#farmersprotest', 824), ('#StandWithFarmers', 813)]


## 5.- Los top 10 users más influyentes en función de lo retweeted de sus tweets.

In [512]:
tweets.head()

,author,create_date,replyCount,hashtags,retweetedTweet,retweetCount,followers
0,ArjunSinghPanam,2009-01-06,0,"[#ModiDontSellFarmers, #FarmersProtest, #FreeN...",None,0,603
1,PrdeepNain,2021-01-29,0,"[#FarmersProtest, #ModiIgnoringFarmersDeaths, ...",None,0,14
2,parmarmaninder,2012-01-27,0,[#FarmersProtest],None,0,72
3,anmoldhaliwal,2010-01-28,0,"[#farmersprotest, #NoFarmersNoFood]",None,0,51
4,KotiaPreet,2021-01-08,0,"[#KisanEktaMorcha, #FarmersProtest, #NoFarmers...",None,0,5


In [569]:
tweets.groupby(by=['author'], sort=True)['retweetCount'].sum().reset_index()

,author,retweetCount
0,0001dchahar,3
1,00143Sahil,0
2,001Sardar,37
3,001harpinder,80
4,007Jashan,0
...,...,...
26514,zsheikh_INC,6
26515,zuberjafri,1
26516,zules7172,0
26517,zulqarnainaazad,2


In [531]:
tweets_aux[(tweets_aux['author']=='sherryontopp')]

KeyError: 'author'